In [1]:
import pandas as pd
import ccxt
import talib
import pandas_ta as ta


In [2]:
def fetch_data(symbol, timeframe='1m', limit=599):
    exchange = ccxt.huobi()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df


In [3]:
def generate_signals(data):
    # Calculate MACD
    data['macd_red'], data['macd_signal_blue'], candle_macd = talib.MACD(data['Close'], fastperiod=34, slowperiod=144, signalperiod=9)

    # Calculate RSI
    data['rsi'] = talib.RSI(data['Close'])

    # Calculate Bollinger Bands
    data['upper_band'], data['middle_band'], data['lower_band'] = talib.BBANDS(data['Close'])

    # Calculate Donchian Channel
    data['upper_dc'], data['lower_dc'] = talib.MAX(data['High'], timeperiod=20), talib.MIN(data['Low'], timeperiod=20)

    return data


In [4]:
def analyze_signals(data):
    signals = pd.DataFrame(index=data.index)
    
    # MACD signals
    signals['macd_buy'] = (data['macd_red'] > data['macd_signal_blue'])
    signals['macd_sell'] = (data['macd_red'] < data['macd_signal_blue']) 

    # RSI signals
    signals['rsi_overbought'] = (data['rsi'] > 70)
    signals['rsi_oversold'] = (data['rsi'] < 30)
    signals['rsi_middle'] = (data['rsi'] > 30) & (data['rsi'] < 70)

    # Bollinger Bands signals
    signals['bollinger_buy'] = data['Close'] < data['lower_band']
    signals['bollinger_sell'] = data['Close'] > data['upper_band']
    signals["bollinger_middle_line_buy"] = (data['middle_band'] < data['Close'])  
    signals["bollinger_middle_line_sell"] = (data['middle_band'] > data['Close'])

    # Donchian Channel signals
    signals['donchian_buy'] = data['Close'] > data['upper_dc']
    signals['donchian_sell'] = data['Close'] < data['lower_dc']
    signals['donchian_buy_middle'] = data['Close'] > data['upper_dc']
    signals['donchian_sell_middle'] = data['Close'] < data['lower_dc']
    
    # Print only the last row of the signals DataFrame
    print(signals.iloc[-1])
    
    return signals


In [5]:
# Example Usage
symbol = 'BTC/USDT'
data = fetch_data(symbol)
data = generate_signals(data)
display(data)
signals = analyze_signals(data)


,Open,High,Low,Close,Volume,macd_red,macd_signal_blue,rsi,upper_band,middle_band,lower_band,upper_dc,lower_dc
timestamp,,,,,,,,,,,,,
2024-02-25 10:02:00,51649.83,51653.33,51649.83,51653.33,0.018668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-25 10:03:00,51653.33,51683.36,51653.32,51675.80,0.603391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-25 10:04:00,51662.99,51662.99,51644.40,51644.40,0.751266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-25 10:05:00,51644.41,51644.41,51641.92,51641.92,0.067132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-25 10:06:00,51641.92,51652.12,51641.92,51652.12,0.529752,NaN,NaN,NaN,51677.449392,51653.514,51629.578608,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-25 19:56:00,51715.30,51715.30,51698.70,51698.70,0.058658,83.005472,81.471828,54.601964,51721.646969,51693.286,51664.925031,51734.83,51664.19
2024-02-25 19:57:00,51698.70,51698.70,51690.00,51690.01,0.453675,82.682333,81.713929,52.292366,51720.349405,51695.630,51670.910595,51734.83,51664.19
2024-02-25 19:58:00,51676.64,51676.64,51667.46,51671.30,0.307516,81.505619,81.672267,47.621771,51721.826533,51690.530,51659.233467,51734.83,51664.19


macd_buy                      False
macd_sell                      True
rsi_overbought                False
rsi_oversold                  False
rsi_middle                     True
bollinger_buy                 False
bollinger_sell                False
bollinger_middle_line_buy     False
bollinger_middle_line_sell     True
donchian_buy                  False
donchian_sell                 False
donchian_buy_middle           False
donchian_sell_middle          False
Name: 2024-02-25 20:00:00, dtype: bool
